In [1]:
import argparse
import os
import sys
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import torch
from sklearn.model_selection import KFold
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, ".."))
sys.path.append(parent_dir)

In [2]:
%load_ext autoreload
%autoreload 2

from drGT import drGT
from drGT.load_data import load_data
from drGT.metrics import evaluate_predictions
from drGT.myutils import get_all_edges_and_labels, get_model_params
from drGT.sampler import BalancedSampler

/Users/inouey2/code/drGAT/.venv/lib/python3.10/site-packages/pubchempy.py:563: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self.charge is not 0:


In [3]:
method = "Transformer"
data = "nci"
task = "test1"
params = get_model_params(task, data, method)

In [4]:
# Load data
(
    drugAct,
    null_mask,
    S_d,
    S_c,
    S_g,
    _,
    _,
    _,
    A_cg,
    A_dg,
) = load_data(data, is_zero_pad=params["is_zero_pad"], verbose=True)

# Update parameters
params.update(
    {
        "n_drug": S_d.shape[0],
        "n_cell": S_c.shape[0],
        "n_gene": S_g.shape[0],
        "gnn_layer": method,
    }
)

load nci
unique drugs	976
unique cells	59
unique drug response	14312
n sensitive	7050
n resistant	7262
AVG Drug binary ratio	0.485
AVG Cell binary ratio	0.497
Over 10 entries (drugs)	853
Over 10 entries (cells)	59
dtis	572
unique drugs	191
unique genes	242
Top 90% variable genes: 	2247
Total selected genes: 	2489
Done!


In [5]:
all_edges, all_labels = get_all_edges_and_labels(drugAct, null_mask)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for train_idx, test_idx in tqdm(kf.split(all_edges)):
    sampler = BalancedSampler(
        drugAct,
        all_edges,
        all_labels,
        train_idx,
        test_idx,
        null_mask,
        S_d,
        S_c,
        S_g,
        A_cg,
        A_dg,
    )

    break

0it [00:00, ?it/s]


In [6]:
probs, true_labels, attention = drGT.predict("best_model.pt", sampler, params)

Using device for prediction: cpu


In [7]:
evaluate_predictions(true_labels, probs)

,Score
Metric,
accuracy,0.8655
f1_score,0.8580
auroc,0.9411
aupr,0.9387
